In [20]:
import sys
from io import BytesIO
from pathlib import Path

# racine du projet
sys.path.append(str(Path().resolve().parent))

from dashboard.src.i_audio import audio_pipeline
from dashboard.src.i_video import video_pipeline
from dashboard.src.o_texte import transcription, description

In [21]:
def path_to_bytesio(path: str | Path) -> BytesIO:
    path = Path(path)
    with path.open("rb") as f:
        buffer = BytesIO(f.read())
    buffer.name = path.name
    buffer.seek(0)
    return buffer

In [22]:
path = "src/input.mp4"
filename = Path(path).name
video_converted = video_pipeline(path_to_bytesio(path))
audio_converted = audio_pipeline(path_to_bytesio(path))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


### ↖ génère les variables du Streamlit (video_converted, audio_converted)
___
### Nouvelles fonctions 

In [59]:
def seconds_to_timecode(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    return f"{hours:02}:{minutes:02}:{secs:06.3f}"

In [62]:
audio_highlighted = [
    {
        "type": "audio",
        "start": seconds_to_timecode(e.get("start")),
        "end": seconds_to_timecode(e.get("end")),
        "text": e.get("text").strip(),
    }
    for e in audio_converted
]

video_highlighted = [
    {
        "type": "video",
        "start": e.get("start").get_timecode(),
        "end": e.get("end").get_timecode(),
        "text": e.get("text").upper(),
    }
    for e in video_converted
]

media_highlighted = video_highlighted + audio_highlighted
media_highlighted.sort(key=lambda x: x["start"])

In [63]:
print(len(video_highlighted), video_highlighted[-1])
print(len(audio_highlighted), audio_highlighted[-1])
print(len(media_highlighted), media_highlighted[-1])

98 {'type': 'video', 'start': '00:02:24.000', 'end': '00:02:44.000', 'text': 'A RED AND WHITE BACKGROUND WITH THE WORDS VOUS'}
56 {'type': 'audio', 'start': '00:02:17.620', 'end': '00:02:17.980', 'text': 'On va.'}
154 {'type': 'video', 'start': '00:02:24.000', 'end': '00:02:44.000', 'text': 'A RED AND WHITE BACKGROUND WITH THE WORDS VOUS'}


In [90]:
from docx import Document
from docx.shared import Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml.ns import qn


def create_styles(document):
    styles = document.styles

    # =========================
    # VIDEO TIMESTAMP STYLE
    # =========================
    style = styles.add_style("VIDEO_TIMESTAMP", WD_STYLE_TYPE.PARAGRAPH)
    style.font.name = "Courier New"
    style.font.size = Pt(8)
    style.paragraph_format.space_after = Pt(6)

    # =========================
    # VIDEO TITLE STYLE
    # =========================
    style = styles.add_style("VIDEO_TITLE", WD_STYLE_TYPE.PARAGRAPH)
    style.font.name = "Courier New"
    style.font.size = Pt(12)
    style.font
    style.font.bold = True

    # =========================
    # CHARACTER PLACEHOLDER
    # =========================
    style = styles.add_style("CHARACTER_PLACEHOLDER", WD_STYLE_TYPE.PARAGRAPH)
    style.font.name = "Courier New"
    style.font.size = Pt(12)
    style.font.bold = True
    style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

    # =========================
    # AUDIO DIALOGUE STYLE
    # =========================
    style = styles.add_style("AUDIO_DIALOGUE", WD_STYLE_TYPE.PARAGRAPH)
    style.font.name = "Courier New"
    style.font.size = Pt(12)
    style.paragraph_format.left_indent = Cm(2)
    style.paragraph_format.right_indent = Cm(2)
    style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY


def set_cinema_margins(document):
    section = document.sections[0]
    section.top_margin = Cm(2.54)
    section.bottom_margin = Cm(2.54)
    section.left_margin = Cm(2.54)
    section.right_margin = Cm(2.54)


def export_scenario(json_data, output_path="scenario.docx"):

    document = Document()

    # Marges cinéma standard
    set_cinema_margins(document)

    # Création des styles personnalisés
    create_styles(document)

    video_counter = 1

    for index, item in enumerate(json_data):
        # Bordure
        next_item = json_data[index + 1]["type"] if index + 1 < len(json_data) else None

        # =========================
        # VIDEO BLOCK
        # =========================
        if item["type"] == "video" and next_item != "video":

            # Numérotation automatique
            timestamp_text = f"{video_counter:03d}  {item['start']} - {item['end']}"
            video_counter += 1

            # Timestamp
            p_time = document.add_paragraph(timestamp_text, style="VIDEO_TIMESTAMP")
            p_time.alignment = WD_ALIGN_PARAGRAPH.LEFT

            # Title
            p_text = document.add_paragraph(item["text"], style="VIDEO_TITLE")
            p_text.alignment = WD_ALIGN_PARAGRAPH.LEFT

        # =========================
        # AUDIO BLOCK
        # =========================
        elif item["type"] == "audio":

            # Placeholder personnage
            document.add_paragraph("...", style="CHARACTER_PLACEHOLDER")

            # Dialogue
            document.add_paragraph(item["text"], style="AUDIO_DIALOGUE")

            # Space
            document.add_paragraph("")

    document.save(output_path)
    print(f"Document saved to {output_path}")

In [91]:
export_scenario(media_highlighted, output_path="src/scenario.docx")

Document saved to src/scenario.docx


In [ ]:
def script(video_converted, audio_converted, filename):
    return None #video_converted + audio_converted + filename